In [1]:
import random, gc, os, pickle, csv, time, re

import datasets.utils
import models.utils
from models.cls_oml_ori_v2 import OML
from models.base_models_ori import LabelAwareReplayMemory

import numpy as np

import higher
import torch
import torch.nn.functional as F
from torch.utils import data

# Ablation 1: No Sort Adapt Score
This Ablation focuses on no sorting of adapt score. So during query should be random sort.

There are a total of 4 changes:
1. Training cls_oml_ori_v2.py: Random Score on Sample Support (no_support_priority=False)
2. Training cls_oml_ori_v2.py: No memory write of update adapt score (memory.update_meta)
3. Memory base_models_ori.py: Write filter condition don’t care (sorted_support >= 1)
4. Testing: No Sample Score (sort_score=False)


# Constants

In [2]:
dataset_order_mapping = {
    1: [2, 0, 3, 1, 4],
    2: [3, 4, 0, 1, 2],
    3: [2, 4, 1, 3, 0],
    4: [0, 2, 1, 4, 3]
}
n_classes = 33
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')


BASE_MODEL_PATH = "/data/model_runs/original_oml/aMEL-nocurrireplay-order4-v2"
find_path = "id4"

model_name = [x for x in os.listdir(BASE_MODEL_PATH) if  re.search(f"{find_path}.*\.pt$", x)][0]
model_path = os.path.join(BASE_MODEL_PATH, model_name)
memory_name = [x for x in os.listdir(BASE_MODEL_PATH) if  re.search(f"{find_path}.*memory\.pickle$", x)][0]
memory_path = os.path.join(BASE_MODEL_PATH, memory_name)
print(f"Using model from {model_path}")
print(f"Using memory_path from {memory_path}")

use_db_cache = True
cache_dir = 'tmp'

Using model from /data/model_runs/original_oml/aMEL-nocurrireplay-order4-v2/OML-order4-id4-2023-11-19_07-33-09.184690.pt
Using memory_path from /data/model_runs/original_oml/aMEL-nocurrireplay-order4-v2/OML-order4-id4-2023-11-19_07-33-09.184721_memory.pickle


In [3]:
args = {
    "order": 1,
    "n_epochs": 1,
    "lr": 3e-5,
    "inner_lr": 0.001*5,
    "meta_lr": 3e-5,
    "model": "roberta",
    "learner": "oml",
    "mini_batch_size": 16,
    "updates": 5*1,
    "write_prob": 0.1,
    "max_length": 448,
    "seed": 42,
    "replay_rate": 0.01,
    "replay_every": 8000,
    "task_aware": True,
    "reverse_support": True,
    "curriculum_replay": True,
    "pln": "1fc"
}
sort_score = True
updates = args["updates"]
mini_batch_size = args["mini_batch_size"]
order = args["order"]

In [4]:
torch.manual_seed(args["seed"])
random.seed(args["seed"])
np.random.seed(args["seed"])

# Load Dataset

In [5]:
print('Loading the datasets')
test_datasets = []
for dataset_id in dataset_order_mapping[order]:
    test_dataset_file = os.path.join(cache_dir, f"{dataset_id}.cache")
    if os.path.exists(test_dataset_file):
        with open(test_dataset_file, 'rb') as f:
            test_dataset = pickle.load(f)
    else:
        test_dataset = datasets.utils.get_dataset_test("", dataset_id)
        print('Loaded {}'.format(test_dataset.__class__.__name__))
        test_dataset = datasets.utils.offset_labels(test_dataset)
        pickle.dump(test_dataset, open( test_dataset_file, "wb" ), protocol=pickle.HIGHEST_PROTOCOL)
        print(f"Pickle saved at {test_dataset_file}")
    test_datasets.append(test_dataset)
print('Finished loading all the datasets')

Loading the datasets
Finished loading all the datasets


# Load Model

In [6]:
learner = OML(device=device, n_classes=n_classes, **args)
print('Using {} as learner'.format(learner.__class__.__name__))
learner.load_model(model_path)
with open(memory_path, 'rb') as f:
#     learner.memory = pickle.load(f)
    memory_buffer = pickle.load(f)


2023-11-19 10:04:04,412 - transformers.tokenization_utils_base - INFO - loading file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-vocab.json from cache at /root/.cache/torch/transformers/d0c5776499adc1ded22493fae699da0971c1ee4c2587111707a4d177d20257a2.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b
2023-11-19 10:04:04,415 - transformers.tokenization_utils_base - INFO - loading file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-merges.txt from cache at /root/.cache/torch/transformers/b35e7cd126cd4229a746b5d5c29a749e8e84438b14bcdb575950584fe33207e8.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda
2023-11-19 10:04:05,714 - transformers.configuration_utils - INFO - loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-config.json from cache at /root/.cache/torch/transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.117c81977c5979de8c088352e74ec6e70f5c66096c2

Using OML as learner


In [7]:
# Setting up task dict for task-aware
memory_buffer.task_dict = {
    0: list(range(5, 9)), # AG
    1: list(range(0, 5)), # Amazon
    2: list(range(0, 5)), # Yelp
    3: list(range(9, 23)), # DBPedia
    4: list(range(23, 33)), # Yahoo
}

In [8]:
dataclass_mapper = {
    "AGNewsDataset": 0,
    "AmazonDataset": 1,
    "YelpDataset": 2,
    "DBPediaDataset": 3,
    "YahooAnswersDataset": 4
}
dataclass_mapper["AGNewsDataset"]

0

# Testing

Select specific column index per row
https://stackoverflow.com/questions/23435782/numpy-selecting-specific-column-index-per-row-by-using-a-list-of-indexes

In [9]:
def evaluate(dataloader, updates, mini_batch_size, dataname=""):
    learner.rln.eval()
    learner.pln.train()
    
    all_losses, all_predictions, all_labels, all_label_conf = [], [], [], []
    all_adaptation_time = []
    # Get Query set first. and then find supporting support set
    for query_idx, (query_text, query_labels) in enumerate(dataloader):
        print(f"Query ID {query_idx}/{len(dataloader)}")
        # The task id to optimize to for support set
        # task_idx = get_task_from_label_list(query_labels, memory_buffer.task_dict)
        task_idx = dataclass_mapper[dataname]
        
    
        support_set = []
        for _ in range(updates):
            text, labels = memory_buffer.read_batch_task(batch_size=mini_batch_size, task_idx=task_idx, sort_score=sort_score)
            support_set.append((text, labels))

        with higher.innerloop_ctx(learner.pln, learner.inner_optimizer,
                                  copy_initial_weights=False, track_higher_grads=False) as (fpln, diffopt):
            
            INNER_tic = time.time()
            # Inner loop
            task_predictions, task_labels = [], []
            support_loss = []
            for text, labels in support_set:
                labels = torch.tensor(labels).to(device)
                input_dict = learner.rln.encode_text(text)
                _repr = learner.rln(input_dict)
                output = fpln(_repr)
                loss = learner.loss_fn(output, labels)
                diffopt.step(loss)
                pred = models.utils.make_prediction(output.detach())
                support_loss.append(loss.item())
                task_predictions.extend(pred.tolist())
                task_labels.extend(labels.tolist())
            INNER_toc = time.time() - INNER_tic
            all_adaptation_time.append(INNER_toc)

            if updates > 0: # You can remove this if, but then it's just ugly errors.
                acc, prec, rec, f1 = models.utils.calculate_metrics(task_predictions, task_labels)
                print('Support set metrics: Loss = {:.4f}, accuracy = {:.4f}, precision = {:.4f}, '
                            'recall = {:.4f}, F1 score = {:.4f}'.format(np.mean(support_loss), acc, prec, rec, f1))

            # Query set is now here!
            query_labels = torch.tensor(query_labels).to(device)
            query_input_dict = learner.rln.encode_text(query_text)
            with torch.no_grad():
                query_repr = learner.rln(query_input_dict)
                query_output = fpln(query_repr) # Output has size of torch.Size([16, 33]) [BATCH, CLASSES]
                query_loss = learner.loss_fn(query_output, query_labels)
            query_loss = query_loss.item()
            # print(output.detach().size())
            # output.detach().max(-1) max on each Batch, which will return [0] max, [1] indices
            query_output_softmax = F.softmax(query_output, -1)
            query_label_conf = query_output_softmax[np.arange(len(query_output_softmax)), query_labels] # Select labels in the softmax of 33 classes

            query_pred = models.utils.make_prediction(query_output.detach())
            query_acc, query_prec, query_rec, query_f1 = models.utils.calculate_metrics(query_pred.tolist(), query_labels.tolist())
            
            print('Query set metrics: Loss = {:.4f}, accuracy = {:.4f}, precision = {:.4f}, '
                'recall = {:.4f}, F1 score = {:.4f}'.format(np.mean(query_loss), query_acc, query_prec, query_rec, query_f1))

            all_losses.append(query_loss)
            all_predictions.extend(query_pred.tolist())
            all_labels.extend(query_labels.tolist())
            all_label_conf.extend(query_label_conf.tolist())

    acc, prec, rec, f1 = models.utils.calculate_metrics(all_predictions, all_labels)
    print('Test metrics: Loss = {:.4f}, accuracy = {:.4f}, precision = {:.4f}, recall = {:.4f}, '
                'F1 score = {:.4f}'.format(np.mean(all_losses), acc, prec, rec, f1))
    return acc, prec, rec, f1, all_predictions, all_labels, all_label_conf, all_adaptation_time

In [10]:
tic = time.time()
print('----------Testing on test set starts here----------')

accuracies, precisions, recalls, f1s = [], [], [], []
all_adapt_time = []
# Data for Visualization: [data_idx, label, label_conf, pred]
data_for_visual = []

for test_dataset in test_datasets:
    print('Testing on {}'.format(test_dataset.__class__.__name__))
    test_dataloader = data.DataLoader(test_dataset, batch_size=mini_batch_size, shuffle=False,
                                      collate_fn=datasets.utils.batch_encode)
    acc, prec, rec, f1, all_pred, all_label, all_label_conf, all_adaptation_time = evaluate(dataloader=test_dataloader, updates=updates, 
                                                mini_batch_size=mini_batch_size, dataname=test_dataset.__class__.__name__)
    
    data_ids = [test_dataset.__class__.__name__ + str(i) for i in range(len(all_label))]
    data_for_visual.extend(list(zip(data_ids, all_label, all_label_conf, all_pred)))
    all_adapt_time.extend(all_adaptation_time)
#     print(data_ids)
#     print(all_label)
#     raise Exception("BREAKPOINT")
    
    accuracies.append(acc)
    precisions.append(prec)
    recalls.append(rec)
    f1s.append(f1)


print()
print("COPY PASTA - not really but ok")
for row in accuracies:
    print(row)
print()
print('Overall test metrics: Accuracy = {:.4f}, precision = {:.4f}, recall = {:.4f}, '
            'F1 score = {:.4f}'.format(np.mean(accuracies), np.mean(precisions), np.mean(recalls), np.mean(f1s)))

toc = time.time() - tic
print(f"Total Time used: {toc//60} minutes")

----------Testing on test set starts here----------
Testing on YelpDataset
Query ID 0/475
Support set metrics: Loss = 0.6490, accuracy = 0.7500, precision = 0.7697, recall = 0.7467, F1 score = 0.7395
Query set metrics: Loss = 0.6042, accuracy = 0.7500, precision = 0.8133, recall = 0.7267, F1 score = 0.7367
Query ID 1/475
Support set metrics: Loss = 0.5319, accuracy = 0.7625, precision = 0.7958, recall = 0.7633, F1 score = 0.7582
Query set metrics: Loss = 0.8154, accuracy = 0.8125, precision = 0.7667, recall = 0.8933, F1 score = 0.7911
Query ID 2/475
Support set metrics: Loss = 0.5792, accuracy = 0.7125, precision = 0.7230, recall = 0.7100, F1 score = 0.6938
Query set metrics: Loss = 0.8908, accuracy = 0.6250, precision = 0.7000, recall = 0.6333, F1 score = 0.6148
Query ID 3/475
Support set metrics: Loss = 0.5285, accuracy = 0.7625, precision = 0.7655, recall = 0.7533, F1 score = 0.7440
Query set metrics: Loss = 1.1741, accuracy = 0.6250, precision = 0.5267, recall = 0.6333, F1 score = 

Support set metrics: Loss = 0.5677, accuracy = 0.7375, precision = 0.7434, recall = 0.7333, F1 score = 0.7236
Query set metrics: Loss = 1.0895, accuracy = 0.6250, precision = 0.8071, recall = 0.6167, F1 score = 0.6364
Query ID 36/475
Support set metrics: Loss = 0.6017, accuracy = 0.7625, precision = 0.8013, recall = 0.7600, F1 score = 0.7621
Query set metrics: Loss = 0.9188, accuracy = 0.6250, precision = 0.6000, recall = 0.6333, F1 score = 0.5600
Query ID 37/475
Support set metrics: Loss = 0.5945, accuracy = 0.7375, precision = 0.7506, recall = 0.7300, F1 score = 0.7195
Query set metrics: Loss = 0.8023, accuracy = 0.8125, precision = 0.9100, recall = 0.8533, F1 score = 0.8592
Query ID 38/475
Support set metrics: Loss = 0.6026, accuracy = 0.6750, precision = 0.6975, recall = 0.6667, F1 score = 0.6590
Query set metrics: Loss = 1.0120, accuracy = 0.6875, precision = 0.6200, recall = 0.6600, F1 score = 0.6378
Query ID 39/475
Support set metrics: Loss = 0.5417, accuracy = 0.7625, precision

Support set metrics: Loss = 0.6702, accuracy = 0.7000, precision = 0.7205, recall = 0.6967, F1 score = 0.6963
Query set metrics: Loss = 1.0491, accuracy = 0.6250, precision = 0.5733, recall = 0.5333, F1 score = 0.5410
Query ID 72/475
Support set metrics: Loss = 0.4877, accuracy = 0.8125, precision = 0.8244, recall = 0.8100, F1 score = 0.8004
Query set metrics: Loss = 1.0981, accuracy = 0.6875, precision = 0.6867, recall = 0.7700, F1 score = 0.7018
Query ID 73/475
Support set metrics: Loss = 0.5983, accuracy = 0.7250, precision = 0.7356, recall = 0.7233, F1 score = 0.7204
Query set metrics: Loss = 0.8871, accuracy = 0.6875, precision = 0.6167, recall = 0.5933, F1 score = 0.5835
Query ID 74/475
Support set metrics: Loss = 0.5966, accuracy = 0.7125, precision = 0.7166, recall = 0.7067, F1 score = 0.6898
Query set metrics: Loss = 1.2238, accuracy = 0.5000, precision = 0.6267, recall = 0.6000, F1 score = 0.5689
Query ID 75/475
Support set metrics: Loss = 0.5763, accuracy = 0.7750, precision

Support set metrics: Loss = 0.6046, accuracy = 0.7375, precision = 0.7695, recall = 0.7267, F1 score = 0.7239
Query set metrics: Loss = 1.3917, accuracy = 0.5000, precision = 0.5333, recall = 0.6000, F1 score = 0.5308
Query ID 107/475
Support set metrics: Loss = 0.6413, accuracy = 0.7375, precision = 0.7551, recall = 0.7300, F1 score = 0.7271
Query set metrics: Loss = 1.1423, accuracy = 0.6250, precision = 0.5100, recall = 0.6000, F1 score = 0.5435
Query ID 108/475
Support set metrics: Loss = 0.6287, accuracy = 0.7000, precision = 0.7066, recall = 0.6967, F1 score = 0.6872
Query set metrics: Loss = 1.2517, accuracy = 0.4375, precision = 0.4200, recall = 0.4067, F1 score = 0.3881
Query ID 109/475
Support set metrics: Loss = 0.5370, accuracy = 0.7500, precision = 0.7640, recall = 0.7500, F1 score = 0.7444
Query set metrics: Loss = 1.0129, accuracy = 0.7500, precision = 0.7500, recall = 0.8033, F1 score = 0.7581
Query ID 110/475
Support set metrics: Loss = 0.5239, accuracy = 0.7375, preci

Support set metrics: Loss = 0.4716, accuracy = 0.8125, precision = 0.8178, recall = 0.8100, F1 score = 0.8034
Query set metrics: Loss = 1.1442, accuracy = 0.5625, precision = 0.6167, recall = 0.5500, F1 score = 0.5576
Query ID 142/475
Support set metrics: Loss = 0.5699, accuracy = 0.7625, precision = 0.7863, recall = 0.7600, F1 score = 0.7486
Query set metrics: Loss = 0.6262, accuracy = 0.7500, precision = 0.6400, recall = 0.7000, F1 score = 0.6667
Query ID 143/475
Support set metrics: Loss = 0.5376, accuracy = 0.8125, precision = 0.8460, recall = 0.8100, F1 score = 0.8133
Query set metrics: Loss = 0.7378, accuracy = 0.6250, precision = 0.7033, recall = 0.6700, F1 score = 0.6267
Query ID 144/475
Support set metrics: Loss = 0.5205, accuracy = 0.8000, precision = 0.8227, recall = 0.7933, F1 score = 0.7909
Query set metrics: Loss = 1.2633, accuracy = 0.5625, precision = 0.5000, recall = 0.5167, F1 score = 0.4610
Query ID 145/475
Support set metrics: Loss = 0.5286, accuracy = 0.8250, preci

...

Support set metrics: Loss = 0.8553, accuracy = 0.7625, precision = 0.7922, recall = 0.7600, F1 score = 0.7557
Query set metrics: Loss = 0.5184, accuracy = 0.8750, precision = 0.8958, recall = 0.8750, F1 score = 0.8500
Query ID 313/475
Support set metrics: Loss = 0.8116, accuracy = 0.7750, precision = 0.7912, recall = 0.7700, F1 score = 0.7700
Query set metrics: Loss = 0.6057, accuracy = 0.8125, precision = 0.8333, recall = 0.9083, F1 score = 0.8438
Query ID 314/475
Support set metrics: Loss = 0.5595, accuracy = 0.8750, precision = 0.8708, recall = 0.8600, F1 score = 0.8586
Query set metrics: Loss = 0.5696, accuracy = 0.8125, precision = 0.8095, recall = 0.7738, F1 score = 0.7891
Query ID 315/475
Support set metrics: Loss = 0.8547, accuracy = 0.7250, precision = 0.7357, recall = 0.7200, F1 score = 0.7091
Query set metrics: Loss = 0.9452, accuracy = 0.7500, precision = 1.0000, recall = 0.8095, F1 score = 0.8762
Query ID 316/475
Support set metrics: Loss = 0.7105, accuracy = 0.8250, preci

Support set metrics: Loss = 0.7046, accuracy = 0.8000, precision = 0.8106, recall = 0.8100, F1 score = 0.8029
Query set metrics: Loss = 0.8025, accuracy = 0.8125, precision = 0.7500, recall = 0.7500, F1 score = 0.7292
Query ID 348/475
Support set metrics: Loss = 0.8112, accuracy = 0.7000, precision = 0.6914, recall = 0.6800, F1 score = 0.6610
Query set metrics: Loss = 0.8788, accuracy = 0.6875, precision = 0.5833, recall = 0.6111, F1 score = 0.5630
Query ID 349/475
Support set metrics: Loss = 0.7207, accuracy = 0.8125, precision = 0.8310, recall = 0.8200, F1 score = 0.8089
Query set metrics: Loss = 0.7335, accuracy = 0.7500, precision = 0.7381, recall = 0.7381, F1 score = 0.7197
Query ID 350/475
Support set metrics: Loss = 0.7800, accuracy = 0.7625, precision = 0.7741, recall = 0.7900, F1 score = 0.7725
Query set metrics: Loss = 0.7477, accuracy = 0.7500, precision = 0.7292, recall = 0.7083, F1 score = 0.6833
Query ID 351/475
Support set metrics: Loss = 0.7492, accuracy = 0.8250, preci

Support set metrics: Loss = 0.6483, accuracy = 0.8125, precision = 0.8347, recall = 0.8300, F1 score = 0.8163
Query set metrics: Loss = 0.7053, accuracy = 0.7500, precision = 0.7143, recall = 0.5714, F1 score = 0.6190
Query ID 383/475
Support set metrics: Loss = 0.7490, accuracy = 0.7500, precision = 0.7422, recall = 0.7400, F1 score = 0.7267
Query set metrics: Loss = 0.7721, accuracy = 0.8125, precision = 0.7381, recall = 0.7810, F1 score = 0.7460
Query ID 384/475
Support set metrics: Loss = 0.6666, accuracy = 0.8125, precision = 0.8218, recall = 0.8200, F1 score = 0.8130
Query set metrics: Loss = 1.4687, accuracy = 0.6250, precision = 0.5833, recall = 0.6500, F1 score = 0.6033
Query ID 385/475
Support set metrics: Loss = 0.8377, accuracy = 0.7500, precision = 0.7594, recall = 0.7700, F1 score = 0.7506
Query set metrics: Loss = 1.1995, accuracy = 0.6875, precision = 0.8125, recall = 0.6979, F1 score = 0.7238
Query ID 386/475
Support set metrics: Loss = 0.6569, accuracy = 0.8000, preci

Support set metrics: Loss = 0.5982, accuracy = 0.8250, precision = 0.8184, recall = 0.8100, F1 score = 0.7996
Query set metrics: Loss = 0.7849, accuracy = 0.7500, precision = 0.8000, recall = 0.8000, F1 score = 0.7667
Query ID 418/475
Support set metrics: Loss = 0.7340, accuracy = 0.7875, precision = 0.8136, recall = 0.7800, F1 score = 0.7719
Query set metrics: Loss = 0.9154, accuracy = 0.6875, precision = 0.7500, recall = 0.6979, F1 score = 0.6905
Query ID 419/475
Support set metrics: Loss = 0.7431, accuracy = 0.7750, precision = 0.7746, recall = 0.7700, F1 score = 0.7665
Query set metrics: Loss = 0.8953, accuracy = 0.5000, precision = 0.4722, recall = 0.5000, F1 score = 0.4833
Query ID 420/475
Support set metrics: Loss = 0.8534, accuracy = 0.6875, precision = 0.7042, recall = 0.6800, F1 score = 0.6721
Query set metrics: Loss = 1.2652, accuracy = 0.6875, precision = 0.5185, recall = 0.6111, F1 score = 0.5333
Query ID 421/475
Support set metrics: Loss = 0.6444, accuracy = 0.8250, preci

Support set metrics: Loss = 0.8041, accuracy = 0.8500, precision = 0.8542, recall = 0.8700, F1 score = 0.8544
Query set metrics: Loss = 0.4074, accuracy = 0.9375, precision = 0.9286, recall = 0.9524, F1 score = 0.9238
Query ID 453/475
Support set metrics: Loss = 0.6448, accuracy = 0.8375, precision = 0.8232, recall = 0.8300, F1 score = 0.8228
Query set metrics: Loss = 0.6085, accuracy = 0.8125, precision = 0.9375, recall = 0.8333, F1 score = 0.8500
Query ID 454/475
Support set metrics: Loss = 0.6375, accuracy = 0.8125, precision = 0.8047, recall = 0.8200, F1 score = 0.8051
Query set metrics: Loss = 0.5505, accuracy = 0.8125, precision = 0.7000, recall = 0.8000, F1 score = 0.7333
Query ID 455/475
Support set metrics: Loss = 0.6042, accuracy = 0.8625, precision = 0.8433, recall = 0.8400, F1 score = 0.8398
Query set metrics: Loss = 0.7612, accuracy = 0.7500, precision = 0.7130, recall = 0.7037, F1 score = 0.7026
Query ID 456/475
Support set metrics: Loss = 0.6776, accuracy = 0.8375, preci

In [11]:
_model_path0 = os.path.splitext(model_path)[0]
csv_filename = _model_path0 + "_update"+ str(updates) +"_results.csv" # for selective replay
with open(csv_filename, 'w') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["data_idx", "label", "label_conf", "pred"])
    csv_writer.writerows(data_for_visual)
print(f"Done writing CSV File at {csv_filename}")

Done writing CSV File at /data/model_runs/original_oml/aMEL-nocurrireplay-order4-v2/OML-order4-id4-2023-11-19_07-33-09.184690_update5_results.csv


In [12]:
# Log Time for Inference
_model_path0 = os.path.splitext(model_path)[0]
time_txt_filename = _model_path0 + "_update"+ str(updates) +"_time_inference.csv" 
with open(time_txt_filename, 'w') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["time_id", "time"])
    csv_writer.writerow(["Total Time", f"{toc//60} minutes"])
    csv_writer.writerow(["mean Adapt Time", f"{np.mean(all_adapt_time)} s"])
print(f"Done writing Time CSV File at {time_txt_filename}")

Done writing Time CSV File at /data/model_runs/original_oml/aMEL-nocurrireplay-order4-v2/OML-order4-id4-2023-11-19_07-33-09.184690_update5_time_inference.csv
